In [3]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [6]:
%fetchfile --print main.py

from sevensegmentdisplay import encodeledstring
from machine import I2C, Pin
import time

i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=100000)

tempAmbient, tempIR = 0, 0
tempBad = True
tempTime = 0

def checkCRC(k, mask):  # mask (matching nibble) different in case
    inCrc = 0
    for inData in [k[0], k[1]]:
        data = inCrc ^ inData;
        data <<= 8 
        for i in range(8):
            if (( data & 0x8000 ) != 0 ):
                data = data ^ (0x1070 << 3)
            data = data << 1
        inCrc = data >> 8
    return (inCrc & mask) == (k[2]&mask)


async def readtemps():
    global ntempcount
    while True:
        global tempAmbient, tempIR, tempBad, tempTime
        try:
            ka = i2c.readfrom_mem(0x5a, 0x06, 3)
            ki = i2c.readfrom_mem(0x5a, 0x07, 3)
            bcrcgood = (checkCRC(ka, 0x0F) and checkCRC(ki, 0xF0))
            print(bcrcgood, ka, ki)
            if not bcrcgood:
                raise OSErro

In [279]:
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.simple/umqtt/simple.py lib/umqtt/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.robust/umqtt/robust.py lib/umqtt/

Sent 43 lines (1045 bytes) to lib/umqtt/robust.py.


In [391]:
%sendtofile main.py

import machine, time

# First get the display up with its functions
sck = machine.Pin(14)
miso = machine.Pin(12)
mosi = machine.Pin(13) 
spi = machine.SPI(sck=sck, miso=miso, mosi=mosi, 
                  baudrate=1000000, polarity=0, phase=0, firstbit=machine.SPI.MSB)
cs = machine.Pin(2, machine.Pin.OUT)  # sets the pin to output
cs.value(1)

def swrite(s):
    cs.value(0)
    time.sleep_ms(5)
    spi.write(s)
    cs.value(1)
    time.sleep_ms(5)

# set up the 
swrite(b"\x0C\x00")
swrite(b"\x0F\x00")
swrite(b"\x0B\x07")
swrite(b"\x09\x00")
swrite(b"\x0C\x01")

nums = b"\x7E\x30\x6D\x79\x33\x5B\x5F\x70\x7F\x7B"
alpha = b"\x77\x1F\x0D\x3D\x4F\x47\x7B\x37\x44jk\x0Em\x15\x1D\x67\x73\x05\x5B\x0F\x1Cvwx\x3Bz"
def writestring(s):
    if type(s) == str:
        s = s.encode()
    ep = 0
    for e in range(8):
        ol = (s[ep]  if ep < len(s)  else 32)
        c = 0x80
        
        if ol == 32:
            c = 0x00
        elif 48 <= ol <= 57:
            c = nums[ol - 48]
        elif 97 <= ol <= 122:
            c = alpha[ol - 97]
        if ep+1 < len(s) and s[ep+1] == ".":
            c += 0x80
            ep += 1
        swrite(b"%c%c" % (8-e, c))
        ep += 1


# wifi situation
writestring("cnncting.")
import network

si = network.WLAN(network.STA_IF)
macaddress = "".join("{:02x}".format(x)  for x in si.config("mac"))
si.active(True)
si.connect(b"DoESLiverpool", b"decafbad00")
while not si.isconnected():
    pass
ipnumber = si.ifconfig()[0] 
print(ipnumber)
writestring("cnncted.")


# MQTT technology
from umqtt.robust import MQTTClient

c = MQTTClient("sevenseg", "10.0.29.167")
c.connect()
c.publish(b"sevenseg/on", b"now")

writestring("777  777")
dispmsg = "oooo"
def sub_cb(topic, msg):
    global dispmsg
    print(topic, msg)
    dispmsg = msg
c.set_callback(sub_cb)
c.connect()
c.subscribe(topic=b"sevenseg/show") 

# main loop
import time
currmsg = dispmsg
br = 1
while True:
    c.check_msg()
    if dispmsg != currmsg: 
        currmsg = dispmsg
        writestring(dispmsg)
        c.publish(b"sevenseg/on", b"now")

    time.sleep(0.1)
    swrite(b"\x0A%c" % ((br%8)+7))  # pulse the brightness
    br += 1


Sent 93 lines (2148 bytes) to main.py.


In [335]:
c.subscribe(topic=b"sevenseg/show") 

In [324]:
c.publish(b"button/thing", b"hello")

In [325]:
print(dir(c))

['__class__', '__dict__', '__init__', '__module__', '__qualname__', 'DEBUG', 'connect', 'disconnect', 'log', 'ssl', 'sock', 'port', 'DELAY', 'delay', 'reconnect', 'publish', 'wait_msg', 'client_id', 'server', 'user', 'keepalive', 'ssl_params', 'pid', 'cb', 'pswd', 'lw_topic', 'lw_msg', 'lw_qos', 'lw_retain', '_send_str', '_recv_len', 'set_callback', 'set_last_will', 'ping', 'subscribe', 'check_msg']


In [352]:
import time
currmsg = dispmsg
br = 1
while True:
    c.check_msg()
    if dispmsg != currmsg: 
        currmsg = dispmsg
        writestring(dispmsg)
    time.sleep(0.1)
    swrite(b"\x0A%c" % ((br%8)+7))
    br += 1

b'button/30aea44cafec' b'woke up'
....b'button/sdfs' b'987'
..b'button/sdfs' b'hello'
.............................................................................................ets Jun  8 2016 00:22:57

rst:0x1 (POWERON_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4436
load:0x40078000,len:0
load:0x40078000,len:11816
entry 0x4007a9fc
OSError: [Errno 2] ENOENT
MicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' 